### Fetching Ugandan news data

**Import the `BeautifulSoup` library for web scraping and `requests` for making web requests**

In [20]:
import requests
from bs4 import BeautifulSoup
import datetime

**Setting up `pymongo`, a package used to connect with `MongoDB`, where the news data will be stored**

In [21]:
import pymongo
db_client = pymongo.MongoClient()
news_db = db_client["ug_news"]
col = news_db["news"]

**Define a method for fetching data from the `New Vision` online newspaper**

In [23]:
def fetch_new_vision():
    """Fetch data from the New Vision online nwespaper"""
    
    # Fetch data from the New Vision online coverpage
    url = "https://www.newvision.co.ug"
    news_href = "https://www.newvision.co.ug/new_vision/news/"

    news_request = requests.get(url)
    coverpage = news_request.content
    
    # Pick out the anchor tags to get the links to the actual news articles
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    article_links = soup1.find_all('a')
    
    # Follow each link and fetch the article content
    text = []

    for link in article_links:
        if news_href in link['href']:
            article = requests.get(link['href'])
            article_content = article.content
            soup2 = BeautifulSoup(article_content, 'html5lib')
            body = soup2.find('div', class_='article-content')
            soup3 = BeautifulSoup(article_content, 'html5lib')
            paragraphs = soup3.find_all('p')
            for p in paragraphs:
                text.append(p.get_text())

    final_text = " ".join(text)
    timestamp = datetime.datetime.now()
    news_entry = {timestamp.strftime("%m/%d/%Y, %H:%M:%S"): final_text}
    return(news_entry)

**Define a method for fetching data from the `Daily Monitor` online newspaper**

In [25]:
def fetch_daily_monitor():
    pass

**Call methods for both newspapers to return their news data**

In [32]:
new_vision_news = fetch_new_vision()
daily_monitor_news = fetch_daily_monitor()

In [33]:
new_vision_news

{'05/20/2020, 11:52:02': "\n\t\t\t\t\t\tBy Vision Reporter\n\t\t\t\t\t \n\t\t\t\t\t\tAdded 20th May 2020 10:57 AM\n\t\t\t\t\t \n\t\t\t\tHe, however, noted that transport in the 40 border districts will not be opened, as the Government monitors the situation, given the fact that the major risk points are now the border points.\n\t\t\t \n\t\t\t\t\t\t President Yoweri Museveni. Photo/File\n\t\t\t\t\t He, however, noted that transport in the 40 border districts will not be opened, as the Government monitors the situation, given the fact that the major risk points are now the border points. COVID-19\xa0President Yoweri Museveni yesterday clarified on a raft of measures he released on Monday, saying next week on May 26, shops, hotels, and private cars will be allowed to operate as the Government moves to ease the lockdown. However, Museveni, who was responding to emerging points of clarifications, fronted by citizens, following his Monday address, said public transport and schools to accommo

In [34]:
daily_monitor_news

**Store text in `mongodb` database**

In [35]:
col.insert_one(new_vision_news)

In [37]:
# col.insert_one(daily_monitor_news)